In [4]:
import requests
import json
from tqdm import tqdm
from convokit import Corpus, User, Utterance, download

In [5]:
mh_corpus = Corpus(filename=download("subreddit-mentalhealth"))
mh_corpus.load_info('utterance',['parsed'])

Dataset already exists at /Users/Emilie/.convokit/downloads/subreddit-mentalhealth


In [ ]:
#a = Corpus(filename=download("subreddit-Anxiety"))

In [6]:
e = Corpus(filename=download('subreddit-EatingDisorders'))

Dataset already exists at /Users/Emilie/.convokit/downloads/subreddit-EatingDisorders


In [7]:
all_data = mh_corpus.merge(e)

In [8]:
#https://gist.github.com/xandaschofield/3c4070b2f232b185ce6a09e47b4e7473 

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer as CV
import string

exclude = set(string.punctuation)


def basic_sanitize(in_string):
    '''Returns a very roughly sanitized version of the input string.'''
    return_string = ''.join([ch for ch in in_string if ord(ch) < 128 and ch not in exclude]).lower()
    return_string = ' '.join(return_string.split())
    return return_string

def bayes_compare_language(l1, l2, ngram=1, prior=.01, cv=None, sig_val=2.573):
    '''
    Arguments:
    - l1, l2; a list of strings from each language sample
    - ngram; an int describing up to what n gram you want to consider (1 is unigrams,
    2 is bigrams + unigrams, etc). Ignored if a custom CountVectorizer is passed.
    - prior; either a float describing a uniform prior, or a vector describing a prior
    over vocabulary items. If you're using a predefined vocabulary, make sure to specify that
    when you make your CountVectorizer object.
    - cv; a sklearn.feature_extraction.text.CountVectorizer object, if desired.
    Returns:
    - A list of length |Vocab| where each entry is a (n-gram, zscore) tuple.'''
    if cv is None and type(prior) is not float:
        print("If using a non-uniform prior:")
        print("Please also pass a count vectorizer with the vocabulary parameter set.")
        quit()
    l1 = [basic_sanitize(l) for l in l1]
    l2 = [basic_sanitize(l) for l in l2]
    if cv is None:
        cv = CV(decode_error = 'ignore', min_df = 10, max_df = .5, ngram_range=(1,ngram),
                binary = False,
                max_features = 15000)
    counts_mat = cv.fit_transform(l1+l2).toarray()
    # Now sum over languages...
    vocab_size = len(cv.vocabulary_)
    print("Vocab size is {}".format(vocab_size))
    if type(prior) is float:
        priors = np.array([prior for i in range(vocab_size)])
    else:
        priors = prior
    z_scores = np.empty(priors.shape[0])
    count_matrix = np.empty([2, vocab_size], dtype=np.float32)
    count_matrix[0, :] = np.sum(counts_mat[:len(l1), :], axis = 0)
    count_matrix[1, :] = np.sum(counts_mat[len(l1):, :], axis = 0)
    a0 = np.sum(priors)
    n1 = 1.*np.sum(count_matrix[0,:])
    n2 = 1.*np.sum(count_matrix[1,:])
    print("Comparing language...")
    for i in range(vocab_size):
        #compute delta
        term1 = np.log((count_matrix[0,i] + priors[i])/(n1 + a0 - count_matrix[0,i] - priors[i]))
        term2 = np.log((count_matrix[1,i] + priors[i])/(n2 + a0 - count_matrix[1,i] - priors[i]))        
        delta = term1 - term2
        #compute variance on delta
        var = 1./(count_matrix[0,i] + priors[i]) + 1./(count_matrix[1,i] + priors[i])
        #store final score
        z_scores[i] = delta/np.sqrt(var)
    index_to_term = {v: k for k, v in cv.vocabulary_.items()}
    sorted_indices = np.argsort(z_scores)
    return_list = [(index_to_term[i], z_scores[i]) for i in sorted_indices]
    
    x_vals = count_matrix.sum(axis=0)
    y_vals = z_scores
    sizes = abs(z_scores) * 2
    neg_color, pos_color, insig_color = ('orange', 'purple', 'grey')
    colors = []
    annots = []
    for i, y in enumerate(y_vals):
        if y > sig_val:
            colors.append(pos_color)
            annots.append(index_to_term[i])
        elif y < -sig_val:
            colors.append(neg_color)
            annots.append(index_to_term[i])
        else:
            colors.append(insig_color)
            annots.append(None)

    fig, ax = plt.subplots()
    ax.scatter(x_vals, y_vals, c=colors, s=sizes, linewidth=0)
    for i, annot in enumerate(annots):
        if annot is not None:
            ax.annotate(annot, (x_vals[i], y_vals[i]), color=colors[i], size=sizes[i])
    ax.set_xscale('log')
    
    plt.savefig('test.pdf')

In [9]:
from nltk.corpus import stopwords
stopwords  = stopwords.words('english')

def removeStopwords(wordlist, stopwords):
    return [w for w in wordlist if w not in stopwords]

def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

In [10]:
mh_convos = [] 
for i in (list(mh_corpus.iter_conversations())):
    mh_convos.append((i.meta['title']))

mh_wordlist = [i.split() for i in mh_convos]
mh_clean_wordlist = removeStopwords(mh_wordlist, stopwords)

In [11]:
e_convos = [] 
for i in (list(e.iter_conversations())):
    e_convos.append((i.meta['title']))

e_wordlist = [i.split() for i in e_convos]
e_clean_wordlist = removeStopwords(e_wordlist, stopwords)

In [12]:
e_convos

['GF has been having stomach problems recently, just yesterday admitted to me she was bulimic for 4yrs before we were together(x-post from r/health)',
 'BED is destroying me.',
 'People Say Stupid Things',
 'Alcoholism + Eating Disorders = Drunkorexia?',
 'Eating Disorders and Holiday Guilt',
 'Help Me Help Make ED Treatment More Accessible to All Who Need It!',
 "I don't know what it's considered but I don't want help with my ED.",
 'New to /r/EatingDisorders. If anyone needs someone to listen, you can PM me.',
 'First World Eating Disorder Problems',
 'What does recovery look like for you?',
 "I don't know what to do...",
 "I don't know what to do",
 'Eating Disorders and the Media: Anti-Airbrush Movement',
 "I'm not sure if I have an eating disorder, some advice?",
 'The long-term effects...',
 "I don't know what to do",
 'ED Sufferer seeks healthy ways to lose weight. Needs to stop the yo-yoing, any advice?',
 "I've been having trouble eating for a little over a year.",
 'Your opin

In [13]:
bayes_compare_language(mh_convos, e_convos)

Vocab size is 413
Comparing language...


Topic Modeling 

In [15]:
#https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/topic_modeling_Gensim.ipynb 
# https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens


[nltk_data] Downloading package wordnet to /Users/Emilie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Emilie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
text_data = []
for line in e_convos: 
    tokens = prepare_text_for_lda(line) 
    text_data.append(tokens)

In [19]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')


## Try 5 topics

In [20]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [21]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.138*"request" + 0.135*"eating" + 0.114*"disorder" + 0.023*"binge"')
(1, '0.076*"request" + 0.027*"recovery" + 0.020*"weight" + 0.019*"still"')
(2, '0.217*"request" + 0.041*"recovery" + 0.024*"friend" + 0.021*"bulimia"')
(3, '0.133*"request" + 0.056*"eating" + 0.039*"disorder" + 0.035*"girlfriend"')
(4, '0.110*"request" + 0.026*"research" + 0.024*"support" + 0.021*"really"')


In [22]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

[(1104, 1)]
[(0, 0.10010921), (1, 0.100147545), (2, 0.10010326), (3, 0.59950674), (4, 0.10013328)]
(0, '0.167*"request" + 0.028*"recovery" + 0.027*"weight" + 0.023*"binge"')
(1, '0.029*"eating" + 0.024*"disorder" + 0.019*"request" + 0.018*"research"')
(2, '0.198*"request" + 0.075*"eating" + 0.065*"disorder" + 0.026*"friend"')


In [23]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.109*"request" + 0.058*"binge" + 0.041*"purging" + 0.028*"years"')
(1, '0.131*"request" + 0.025*"bulimic" + 0.024*"scare" + 0.022*"sister"')
(2, '0.162*"request" + 0.091*"recovery" + 0.040*"question" + 0.033*"relapse"')
(3, '0.247*"request" + 0.060*"girlfriend" + 0.036*"recover" + 0.032*"struggle"')
(4, '0.056*"please" + 0.035*"request" + 0.025*"never" + 0.020*"affect"')
(5, '0.166*"request" + 0.060*"weight" + 0.032*"anorexic" + 0.031*"healthy"')
(6, '0.058*"request" + 0.046*"research" + 0.041*"survey" + 0.030*"study"')
(7, '0.192*"request" + 0.089*"friend" + 0.067*"advice" + 0.022*"trigger"')
(8, '0.204*"eating" + 0.164*"disorder" + 0.149*"request" + 0.021*"binge"')
(9, '0.106*"request" + 0.064*"anyone" + 0.026*"calorie" + 0.022*"experience"')


In [24]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [26]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [27]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [28]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
